<a href="https://colab.research.google.com/github/EvenSol/NeqSim-Colab/blob/master/notebooks/reactions/blue_ammonia_production.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Blue Ammonia Production

Blue ammonia is produced from natural gas with carbon dioxide capture, combining steam methane reforming (SMR) and the Haber–Bosch process.
The main reactions are:

$$\mathrm{CH_4 + H_2O \rightarrow CO + 3H_2}$$
$$\mathrm{CO + H_2O \rightarrow CO_2 + H_2}$$
Overall hydrogen production:
$$\mathrm{CH_4 + 2H_2O \rightarrow CO_2 + 4H_2}$$
Ammonia synthesis (Haber–Bosch):
$$\mathrm{N_2 + 3H_2 \rightleftharpoons 2NH_3}$$

The following example uses NeqSim's Gibbs reactor to simulate these steps.

In [ ]:
%%capture
!pip install neqsim
import neqsim
from neqsim import jneqsim


In [ ]:
# Steam methane reforming to produce hydrogen
system_reformer = jneqsim.thermo.system.SystemSrkEos(1000.0, 30.0)  # 1000 K, 30 bar
system_reformer.addComponent('methane', 1.0)
system_reformer.addComponent('water', 2.0)
system_reformer.addComponent('hydrogen', 0.0)
system_reformer.addComponent('CO', 0.0)
system_reformer.addComponent('CO2', 0.0)
system_reformer.setMixingRule(2)
feed = jneqsim.process.equipment.stream.Stream('feed', system_reformer)
reformer = jneqsim.process.equipment.reactor.GibbsReactor('reformer', feed)
reformer.setUseAllDatabaseSpecies(False)
reformer.setDampingComposition(0.01)
reformer.setMaxIterations(2500)
reformer.setConvergenceTolerance(1e-3)
reformer.setEnergyMode('isothermal')
reformer.run()
out1 = reformer.getOutletStream().getThermoSystem()
for name in ['methane','hydrogen','CO','CO2','water']:
    print(name, round(out1.getComponent(name).getz(),4))

In [ ]:
# Cooling and separating to capture CO2 and water
cooler = jneqsim.process.equipment.heatexchanger.Cooler('cooler', reformer.getOutletStream())
cooler.setOutTemperature(220.0)
cooler.run()
separator = jneqsim.process.equipment.separator.Separator('separator', cooler.getOutStream())
separator.run()
hydrogen_stream = separator.getGasOutStream()
gas = hydrogen_stream.getThermoSystem()
for name in ['hydrogen','CO2','methane']:
    try:
        print(name, round(gas.getComponent(name).getz(),4))
    except:
        pass

In [ ]:
# Ammonia synthesis using Gibbs reactor
system_NH3 = jneqsim.thermo.system.SystemSrkEos(700.0, 150.0)  # 700 K, 150 bar
system_NH3.addComponent('hydrogen', 3.0)
system_NH3.addComponent('nitrogen', 1.0)
system_NH3.addComponent('ammonia', 0.0)
system_NH3.setMixingRule(2)
synthesis = jneqsim.process.equipment.stream.Stream('synthesis gas', system_NH3)
reactorNH3 = jneqsim.process.equipment.reactor.GibbsReactor('ammonia reactor', synthesis)
reactorNH3.setUseAllDatabaseSpecies(False)
reactorNH3.setDampingComposition(0.01)
reactorNH3.setMaxIterations(2500)
reactorNH3.setConvergenceTolerance(1e-3)
reactorNH3.setEnergyMode('isothermal')
reactorNH3.run()
outNH3 = reactorNH3.getOutletStream().getThermoSystem()
for name in ['hydrogen','nitrogen','ammonia']:
    print(name, round(outNH3.getComponent(name).getz(),4))